In [19]:
pip install dash pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [20]:
import dash
from dash import html
from dash import dcc
import pandas as pd
from dash.dependencies import Input, Output
import plotly.express as px

In [21]:
path="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [22]:
df=pd.read_csv(path)

In [23]:
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [24]:
# Initialize the Dash app
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'padding-top': 20}),
    
    # Dropdown menu for selecting launch site
    dcc.Dropdown(id='site-dropdown',
                 options=[{'label': site, 'value': site} for site in ['All Sites'] + list(df['Launch Site'].unique())],
                 value='All Sites',
                 placeholder="Select a launch site",
                 style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}),
    
    # Pie chart to display the success rate
    html.Div(dcc.Graph(id='success-pie-chart')),
])


In [25]:
app

In [26]:
app.run_server(mode='inline')

In [27]:
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'All Sites':
        # Calculate success counts for all launch sites
        success_counts = df['Launch Site'].value_counts()
        print(success_counts) 
        title = f'Success Rate for All SpaceX Launch Sites {selected_site}'
        fig = px.pie(success_counts, values=success_counts.values, names=success_counts.index,
                 title=title)
    
    else:
        # Filter data for the selected launch site
        filtered_df = df[df['Launch Site'] == selected_site]
        success_counts = filtered_df['class'].value_counts()
        success_counts
        title = f'Success Rate for {selected_site}'

    # Create pie chart
    fig = px.pie(success_counts, values=success_counts.values, names=success_counts.index,
                 title=title)

    return fig


In [28]:

    app.run_server(mode='inline')

Launch Site
CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: count, dtype: int64


In [35]:


# Define the app layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'padding-top': 20}),
    
    # Range slider for selecting payload mass
    dcc.RangeSlider(id='payload-slider',
                    min=int(df['Payload Mass (kg)'].min()),
                    max=int(df['Payload Mass (kg)'].max()),
                    step=1000,
                    marks={i: f'{i}' for i in range(int(df['Payload Mass (kg)'].min()), int(df['Payload Mass (kg)'].max())+1, 2000)},
                    value=[int(df['Payload Mass (kg)'].min()), int(df['Payload Mass (kg)'].max())],
                    ),
    
    # Pie chart to display the success rate
    html.Div(dcc.Graph(id='success-pie-chart')),
])

# Define callback function to update pie chart based on dropdown and range slider
@app.callback(
    Output('success-pie-chart', 'figure'),
    [Input('payload-slider', 'value')]
)
def update_pie_chart(selected_site, payload_range):
    if selected_site == 'All Sites':
        filtered_df = df[(df['Payload Mass (kg)'] >= payload_range[0]) & (df['Payload Mass (kg)'] <= payload_range[1])]
        title = f'Success Rate for All SpaceX Launch Sites '
    else:
        filtered_df = df[(df['Launch Site'] == selected_site) & 
                         (df['Payload Mass (kg)'] >= payload_range[0]) & 
                         (df['Payload Mass (kg)'] <= payload_range[1])]
        title = f'Success Rate for {selected_site}'

    # Calculate success and failure counts
    success_counts = filtered_df['class'].value_counts()

    # Create pie chart
    fig = px.pie(success_counts, values=success_counts.values, names=success_counts.index,
                 title=title)

    return fig




In [36]:
app.run_server(mode='inline')

In [31]:

# Define the app layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center', 'padding-top': 20}),
    
    # Dropdown menu for selecting launch site
    dcc.Dropdown(id='site-dropdown',
                 options=[{'label': site, 'value': site} for site in ['All Sites'] + list(df['Launch Site'].unique())],
                 value='All Sites',
                 placeholder="Select a launch site",
                 style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}),
    
    # Range slider for selecting payload mass
    dcc.RangeSlider(id='payload-slider',
                    min=int(df['Payload Mass (kg)'].min()),
                    max=int(df['Payload Mass (kg)'].max()),
                    step=1000,
                    marks={i: f'{i}' for i in range(int(df['Payload Mass (kg)'].min()), int(df['Payload Mass (kg)'].max())+1, 2000)},
                    value=[int(df['Payload Mass (kg)'].min()), int(df['Payload Mass (kg)'].max())],
                    ),
    
    # Scatter plot to display payload vs. launch outcome
    html.Div(dcc.Graph(id='payload-scatter-chart')),
    
    # Pie chart to display the success rate
    html.Div(dcc.Graph(id='success-pie-chart')),
])


In [33]:
@app.callback(
    Output('payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    if selected_site == 'All Sites':
        filtered_df = df[(df['Payload Mass (kg)'] >= payload_range[0]) & (df['Payload Mass (kg)'] <= payload_range[1])]
        title = f'Payload vs. Launch Outcome for All SpaceX Launch Sites'
    else:
        filtered_df = df[(df['Launch Site'] == selected_site) & 
                         (df['Payload Mass (kg)'] >= payload_range[0]) & 
                         (df['Payload Mass (kg)'] <= payload_range[1])]
        title = f'Payload vs. Launch Outcome for {selected_site}'
    
    # Create scatter plot
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                     title=title, hover_data=['Launch Site', 'Payload Mass (kg)', 'class', 'Booster Version'])
    
    fig.update_layout(showlegend=True)
    
    return fig


In [34]:
app.run_server(mode='inline')

Launch Site
CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: count, dtype: int64
